In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# Hyperparameters
input_size = 1
hidden_size = 16
output_size = 2
seq_length = 5

# Define the RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        last_hidden = out[:, -1, :]
        return self.fc(last_hidden)

# Initialize
model = SimpleRNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training data
train_data = [
    (torch.tensor([[[1.], [2.], [3.], [4.], [5.]]]), torch.tensor([1])),  # increasing
    (torch.tensor([[[5.], [4.], [3.], [2.], [1.]]]), torch.tensor([0])),  # decreasing
    (torch.tensor([[[0.], [1.], [2.], [3.], [4.]]]), torch.tensor([1])),
    (torch.tensor([[[9.], [7.], [5.], [3.], [1.]]]), torch.tensor([0]))
]

# Training loop
for epoch in range(100):
    total_loss = 0
    for inputs, labels in train_data:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss:.4f}")

# Function to test on unseen sequence
def predict_sequence(seq):
    with torch.no_grad():
        seq_tensor = torch.tensor(seq, dtype=torch.float32).view(1, seq_length, 1)
        output = model(seq_tensor)
        predicted_class = torch.argmax(output, dim=1).item()
        print(f"Input: {seq} → Predicted class: {predicted_class} ({'Increasing' if predicted_class == 1 else 'Decreasing'})")

# Test on unseen sequences
predict_sequence([2, 4, 6, 8, 10])   # unseen increasing
predict_sequence([9, 6, 4, 2, 0])    # unseen decreasing
predict_sequence([1, 3, 5, 4, 2])    # tricky one


Epoch 0, Loss: 2.5151
Epoch 20, Loss: 0.0079
Epoch 40, Loss: 0.0035
Epoch 60, Loss: 0.0021
Epoch 80, Loss: 0.0015
Input: [2, 4, 6, 8, 10] → Predicted class: 1 (Increasing)
Input: [9, 6, 4, 2, 0] → Predicted class: 0 (Decreasing)
Input: [1, 3, 5, 4, 2] → Predicted class: 1 (Increasing)
